In [1]:
from cryptography.fernet import Fernet
import hashlib
import base64
import os
import secrets


In [2]:
def generate_random_hash_key():
    """Generate a random hash key using secrets."""
    return secrets.token_hex(16)


In [3]:
def generate_encryption_key(hash_key):
    """Generate a SHA-256 encryption key from the hash key."""
    sha256 = hashlib.sha256()
    sha256.update(hash_key.encode('utf-8'))
    return sha256.digest()

In [4]:
def image_to_binary(image_path):
    """Convert an image to binary data and display the result."""
    with open(image_path, 'rb') as image_file:
        binary_data = image_file.read()
    print(f"Image at '{image_path}' has been converted to binary data.")
    print("Binary Data (partial):", binary_data[:100], "...")  # Display the first 100 bytes for brevity
    return binary_data

image_path = "C:/Users/VIGNESH/Downloads/fish.jpg"  
binary_data = image_to_binary(image_path)


Image at 'C:/Users/VIGNESH/Downloads/fish.jpg' has been converted to binary data.
Binary Data (partial): b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x01\x00`\x00`\x00\x00\xff\xdb\x00C\x00\x08\x06\x06\x07\x06\x05\x08\x07\x07\x07\t\t\x08\n\x0c\x14\r\x0c\x0b\x0b\x0c\x19\x12\x13\x0f\x14\x1d\x1a\x1f\x1e\x1d\x1a\x1c\x1c $.\' ",#\x1c\x1c(7),01444\x1f\'9=82<.342\xff\xdb\x00C\x01\t\t\t\x0c\x0b\x0c' ...


In [5]:
def binary_to_image(binary_data, output_path):
    """Convert binary data back to an image file."""
    with open(output_path, 'wb') as image_file:
        image_file.write(binary_data)


In [6]:

def encrypt_binary_data(binary_data):
    """Encrypt binary data using XOR with a generated hash key."""
    # Generate a random hash key
    encryption_key = os.urandom(16)  # 16 bytes for a strong key
    print(f"Generated Hash Key (Hex): {encryption_key.hex()}")  # Display the hash key in hexadecimal format

    # Perform XOR encryption
    encrypted_data = bytearray()
    key_length = len(encryption_key)
    for i, byte in enumerate(binary_data):
        encrypted_data.append(byte ^ encryption_key[i % key_length])
    
    return encrypted_data, encryption_key

# Example binary data
binary_data = b"Example binary data for encryption"

# Call the function correctly with a single argument
encrypted_data, encryption_key = encrypt_binary_data(binary_data)

# Display encrypted data
print("Encrypted Data (partial):", encrypted_data[:100], "...")  # Show the first 100 bytes


Generated Hash Key (Hex): 7991d8cef9b59991c6e47dcb674000b2
Encrypted Data (partial): bytearray(b'<\xe9\xb9\xa3\x89\xd9\xfc\xb1\xa4\x8d\x13\xaa\x159 \xd6\x18\xe5\xb9\xee\x9f\xda\xeb\xb1\xa3\x8a\x1e\xb9\x1e0t\xdb\x16\xff') ...


In [7]:
def decrypt_binary_data(encrypted_data, decryption_key):
    """Decrypt binary data using XOR with the provided decryption key."""
    decrypted_data = bytearray()
    key_length = len(decryption_key)
    for i, byte in enumerate(encrypted_data):
        decrypted_data.append(byte ^ decryption_key[i % key_length])
    return decrypted_data


In [8]:
def sender_workflow(image_path):
    """Sender workflow to convert image to binary and encrypt it."""
    # Convert image to binary
    binary_data = image_to_binary(image_path)
    print("Image at '{}' has been converted to binary data.".format(image_path))
    print("Binary Data (partial):", binary_data[:100], "...")  # Display partial binary data

    # Encrypt binary data (encryption key is generated internally)
    encrypted_data, encryption_key = encrypt_binary_data(binary_data)
    print(f"Generated Hash Key (Hex): {encryption_key.hex()}")  # Display the hash key for sharing securely

    # Save encrypted data to a file
    encrypted_file_path = "encrypted_data.bin"  # Adjust path if necessary
    with open(encrypted_file_path, "wb") as encrypted_file:
        encrypted_file.write(encrypted_data)
    print(f"Encrypted file saved at: {encrypted_file_path}")

    return encrypted_file_path, encryption_key.hex()  # Return file path and hash key as hex


In [9]:
def receiver_workflow(encrypted_file_path, hash_key_hex, output_image_path):
    """Receiver workflow to decrypt binary data and restore the image."""
    # Convert the hexadecimal hash key back to bytes
    decryption_key = bytes.fromhex(hash_key_hex)

    # Read the encrypted binary data
    with open(encrypted_file_path, "rb") as encrypted_file:
        encrypted_data = encrypted_file.read()

    # Decrypt the binary data
    try:
        decrypted_data = decrypt_binary_data(encrypted_data, decryption_key)
        print("Decryption successful! Writing image to:", output_image_path)

        # Write the decrypted binary data to an image file
        with open(output_image_path, "wb") as output_file:
            output_file.write(decrypted_data)
    except Exception as e:
        print("Decryption failed. Error:", e)


In [10]:
if __name__ == "__main__":
    print("--- Sender Workflow ---")
    sender_image_path = "C:/Users/VIGNESH/Downloads/fish.jpg"  # Replace with the path to your image
    encrypted_file, sender_hash_key = sender_workflow(sender_image_path)

    print("\n--- Receiver Workflow ---")
    receiver_hash_key = input("Enter the hash key: ")
    
    receiver_output_image_path = "D:/crypto_outputs/output34.jpeg"  # Path to save the decrypted image
    
    receiver_workflow(encrypted_file, receiver_hash_key, receiver_output_image_path)


--- Sender Workflow ---
Image at 'C:/Users/VIGNESH/Downloads/fish.jpg' has been converted to binary data.
Binary Data (partial): b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x01\x00`\x00`\x00\x00\xff\xdb\x00C\x00\x08\x06\x06\x07\x06\x05\x08\x07\x07\x07\t\t\x08\n\x0c\x14\r\x0c\x0b\x0b\x0c\x19\x12\x13\x0f\x14\x1d\x1a\x1f\x1e\x1d\x1a\x1c\x1c $.\' ",#\x1c\x1c(7),01444\x1f\'9=82<.342\xff\xdb\x00C\x01\t\t\t\x0c\x0b\x0c' ...
Image at 'C:/Users/VIGNESH/Downloads/fish.jpg' has been converted to binary data.
Binary Data (partial): b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x01\x00`\x00`\x00\x00\xff\xdb\x00C\x00\x08\x06\x06\x07\x06\x05\x08\x07\x07\x07\t\t\x08\n\x0c\x14\r\x0c\x0b\x0b\x0c\x19\x12\x13\x0f\x14\x1d\x1a\x1f\x1e\x1d\x1a\x1c\x1c $.\' ",#\x1c\x1c(7),01444\x1f\'9=82<.342\xff\xdb\x00C\x01\t\t\t\x0c\x0b\x0c' ...
Generated Hash Key (Hex): 13c9a7485a0ca2f8c737028f9a4d445c
Generated Hash Key (Hex): 13c9a7485a0ca2f8c737028f9a4d445c
Encrypted file saved at: encrypted_data.bin

--- Receiver Workflow

In [3]:

from cryptography.fernet import Fernet
import hashlib
import os
import secrets
import base64

# Generate a random hash key
def generate_random_hash_key():
    """Generate a random hash key using secrets."""
    return secrets.token_hex(16)

# Generate an encryption key from a hash key
def generate_encryption_key(hash_key):
    """Generate a SHA-256 encryption key from the hash key."""
    sha256 = hashlib.sha256()
    sha256.update(hash_key.encode('utf-8'))
    return base64.urlsafe_b64encode(sha256.digest())

# Convert file to binary
def file_to_binary(file_path):
    """Convert any file to binary data."""
    with open(file_path, 'rb') as file:
        binary_data = file.read()
    print(f"File '{file_path}' has been converted to binary data.")
    return binary_data

# Convert binary to file
def binary_to_file(binary_data, output_path):
    """Convert binary data back to a file."""
    with open(output_path, 'wb') as file:
        file.write(binary_data)
    print(f"Binary data has been written to '{output_path}'.")

# Encrypt binary data
def encrypt_file(file_path, encryption_key, encrypted_output_path):
    """Encrypt the binary data of a file and save the encrypted data."""
    binary_data = file_to_binary(file_path)
    fernet = Fernet(encryption_key)
    encrypted_data = fernet.encrypt(binary_data)
    binary_to_file(encrypted_data, encrypted_output_path)
    print(f"File '{file_path}' has been encrypted and saved as '{encrypted_output_path}'.")

# Decrypt binary data
def decrypt_file(encrypted_file_path, encryption_key, decrypted_output_path):
    """Decrypt encrypted binary data and save the original file."""
    encrypted_data = file_to_binary(encrypted_file_path)
    fernet = Fernet(encryption_key)
    decrypted_data = fernet.decrypt(encrypted_data)
    binary_to_file(decrypted_data, decrypted_output_path)
    print(f"File '{encrypted_file_path}' has been decrypted and saved as '{decrypted_output_path}'.")

# Example usage

if __name__ == "__main__":
    # Paths for input and output files
    file_path = "/home/wac/Downloads/crop_yield.csv"  # Replace with your file path
    encrypted_output_path = "example_encrypted.dat"
    decrypted_output_path = "example_decrypted.csv"

    # Generate keys
    hash_key = generate_random_hash_key()
    encryption_key = generate_encryption_key(hash_key)

    print(f"Generated Hash Key: {hash_key}")
    print(f"Encryption Key: {encryption_key.decode()}\n")

    # Encrypt the file
    encrypt_file(file_path, encryption_key, encrypted_output_path)

    # Decrypt the file
    decrypt_file(encrypted_output_path, encryption_key, decrypted_output_path)


Generated Hash Key: 1b5e3b1b9022b27c074bb679a238621c
Encryption Key: nxC5KHQrVh67GuH7bfqe_9gPmwwN7vemnhvky_lerEI=

File '/home/wac/Downloads/crop_yield.csv' has been converted to binary data.
Binary data has been written to 'example_encrypted.dat'.
File '/home/wac/Downloads/crop_yield.csv' has been encrypted and saved as 'example_encrypted.dat'.
File 'example_encrypted.dat' has been converted to binary data.
Binary data has been written to 'example_decrypted.csv'.
File 'example_encrypted.dat' has been decrypted and saved as 'example_decrypted.csv'.


In [4]:
import base64
from cryptography.fernet import Fernet
import hashlib
import os
import secrets


# Generate a random hash key
def generate_random_hash_key():
    """Generate a random hash key using secrets."""
    return secrets.token_hex(16)


# Generate an encryption key from a hash key
def generate_encryption_key(hash_key):
    """Generate a SHA-256 encryption key from the hash key."""
    sha256 = hashlib.sha256()
    sha256.update(hash_key.encode('utf-8'))
    return base64.urlsafe_b64encode(sha256.digest())


# Convert file to binary
def file_to_binary(file_path):
    """Convert any file to binary data."""
    with open(file_path, 'rb') as file:
        return file.read()


# Convert binary to file
def binary_to_file(binary_data, output_path):
    """Convert binary data back to a file."""
    with open(output_path, 'wb') as file:
        file.write(binary_data)


# Encrypt a file and save the encrypted version
def encrypt_file(file_path, encrypted_output_path):
    """Encrypt the file and save it."""
    # Generate hash key and encryption key
    hash_key = generate_random_hash_key()
    encryption_key = generate_encryption_key(hash_key)

    # Read and encrypt the file
    binary_data = file_to_binary(file_path)
    fernet = Fernet(encryption_key)
    encrypted_data = fernet.encrypt(binary_data)

    # Save the encrypted file
    binary_to_file(encrypted_data, encrypted_output_path)

    # Return the hash key (to be shared with the user)
    print(f"File encrypted successfully! Encrypted file: {encrypted_output_path}")
    print(f"Hash Key (Save this securely!): {hash_key}")


# Decrypt an encrypted file after validating the hash key
def decrypt_file(encrypted_file_path, decrypted_output_path):
    """Decrypt the encrypted file after validating the hash key."""
    # Ask user for the hash key
    hash_key = input("Enter the hash key to decrypt the file: ").strip()

    try:
        # Regenerate the encryption key from the hash key
        encryption_key = generate_encryption_key(hash_key)

        # Read the encrypted file
        encrypted_data = file_to_binary(encrypted_file_path)

        # Decrypt the file
        fernet = Fernet(encryption_key)
        decrypted_data = fernet.decrypt(encrypted_data)

        # Save the decrypted file
        binary_to_file(decrypted_data, decrypted_output_path)
        print(f"File decrypted successfully! Decrypted file: {decrypted_output_path}")
    except Exception as e:
        print("Decryption failed! Invalid hash key or corrupted file.")
        print(f"Error: {e}")


# Example usage
if __name__ == "__main__":
    # Paths for input and output files
    file_path = "/home/wac/Downloads/PY25004.docx"  # Replace with your file path
    encrypted_output_path = "example_encrypted.dat"
    decrypted_output_path = "example_decrypted.docx"

    # Encrypt the file
    encrypt_file(file_path, encrypted_output_path)

    # Decrypt the file
    decrypt_file(encrypted_output_path, decrypted_output_path)


File encrypted successfully! Encrypted file: example_encrypted.dat
Hash Key (Save this securely!): be8fdf58d3c9151a6393626557c52ac8
File decrypted successfully! Decrypted file: example_decrypted.docx


In [5]:
# independent encrypter

import base64
from cryptography.fernet import Fernet
import hashlib
import secrets


# Generate a random hash key
def generate_random_hash_key():
    """Generate a random hash key using secrets."""
    return secrets.token_hex(16)


# Generate an encryption key from a hash key
def generate_encryption_key(hash_key):
    """Generate a SHA-256 encryption key from the hash key."""
    sha256 = hashlib.sha256()
    sha256.update(hash_key.encode('utf-8'))
    return base64.urlsafe_b64encode(sha256.digest())


# Convert file to binary
def file_to_binary(file_path):
    """Convert any file to binary data."""
    with open(file_path, 'rb') as file:
        return file.read()


# Convert binary to file
def binary_to_file(binary_data, output_path):
    """Convert binary data back to a file."""
    with open(output_path, 'wb') as file:
        file.write(binary_data)


# Encrypt a file and save the encrypted version
def encrypt_file(file_path, encrypted_output_path):
    """Encrypt the file and save it."""
    # Generate hash key and encryption key
    hash_key = generate_random_hash_key()
    encryption_key = generate_encryption_key(hash_key)

    # Read and encrypt the file
    binary_data = file_to_binary(file_path)
    fernet = Fernet(encryption_key)
    encrypted_data = fernet.encrypt(binary_data)

    # Save the encrypted file
    binary_to_file(encrypted_data, encrypted_output_path)

    # Return the hash key (to be shared with the user)
    print(f"File encrypted successfully! Encrypted file: {encrypted_output_path}")
    print(f"Hash Key (Save this securely!): {hash_key}")


if __name__ == "__main__":
    # Get user input for the file path
    file_path = input("Enter the path of the file to encrypt: ").strip()
    encrypted_output_path = input("Enter the path to save the encrypted file (e.g., encrypted.dat): ").strip()

    # Encrypt the file
    encrypt_file(file_path, encrypted_output_path)


File encrypted successfully! Encrypted file: encrypted.dat
Hash Key (Save this securely!): af4a30ff6ea22519686a1cb83e69b131


In [6]:
# independent decrypter 
import base64
from cryptography.fernet import Fernet
import hashlib


# Generate an encryption key from a hash key
def generate_encryption_key(hash_key):
    """Generate a SHA-256 encryption key from the hash key."""
    sha256 = hashlib.sha256()
    sha256.update(hash_key.encode('utf-8'))
    return base64.urlsafe_b64encode(sha256.digest())


# Convert file to binary
def file_to_binary(file_path):
    """Convert any file to binary data."""
    with open(file_path, 'rb') as file:
        return file.read()


# Convert binary to file
def binary_to_file(binary_data, output_path):
    """Convert binary data back to a file."""
    with open(output_path, 'wb') as file:
        file.write(binary_data)


# Decrypt an encrypted file after validating the hash key
def decrypt_file(encrypted_file_path, decrypted_output_path):
    """Decrypt the encrypted file after validating the hash key."""
    # Ask user for the hash key
    hash_key = input("Enter the hash key to decrypt the file: ").strip()

    try:
        # Regenerate the encryption key from the hash key
        encryption_key = generate_encryption_key(hash_key)

        # Read the encrypted file
        encrypted_data = file_to_binary(encrypted_file_path)

        # Decrypt the file
        fernet = Fernet(encryption_key)
        decrypted_data = fernet.decrypt(encrypted_data)

        # Save the decrypted file
        binary_to_file(decrypted_data, decrypted_output_path)
        print(f"File decrypted successfully! Decrypted file: {decrypted_output_path}")
    except Exception as e:
        print("Decryption failed! Invalid hash key or corrupted file.")
        print(f"Error: {e}")


if __name__ == "__main__":
    # Get user input for the encrypted file path
    encrypted_file_path = input("Enter the path of the encrypted file: ").strip()
    decrypted_output_path = input("Enter the path to save the decrypted file (e.g., decrypted_output.txt): ").strip()

    # Decrypt the file
    decrypt_file(encrypted_file_path, decrypted_output_path)


File decrypted successfully! Decrypted file: output.jpg


new cloud based encryption and block chain encryption

In [8]:
# independent encrypter
# 

import base64
import hashlib
import secrets
import json
import socket
from cryptography.fernet import Fernet
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from datetime import datetime

# Generate a random hash key
def generate_random_hash_key():
    """Generate a random hash key using secrets."""
    return secrets.token_hex(16)

# Generate an encryption key from a hash key
def generate_encryption_key(hash_key):
    """Generate a SHA-256 encryption key from the hash key."""
    sha256 = hashlib.sha256()
    sha256.update(hash_key.encode('utf-8'))
    return base64.urlsafe_b64encode(sha256.digest())

# Convert file to binary
def file_to_binary(file_path):
    """Convert any file to binary data."""
    with open(file_path, 'rb') as file:
        return file.read()

# Convert binary to file
def binary_to_file(binary_data, output_path):
    """Convert binary data back to a file."""
    with open(output_path, 'wb') as file:
        file.write(binary_data)

# Save blockchain data to JSON file
def save_blockchain_data(data):
    """Save blockchain data to a JSON file."""
    try:
        with open("block_data.json", "r") as file:
            blockchain = json.load(file)
    except FileNotFoundError:
        blockchain = []

    blockchain.append(data)
    with open("block_data.json", "w") as file:
        json.dump(blockchain, file, indent=4)

# Encrypt a file and save the encrypted version
def encrypt_file(file_path, encrypted_output_path):
    """Encrypt the file and save it."""
    # Generate hash key and encryption key
    hash_key = generate_random_hash_key()
    encryption_key = generate_encryption_key(hash_key)

    # Read and encrypt the file
    binary_data = file_to_binary(file_path)
    fernet = Fernet(encryption_key)
    encrypted_data = fernet.encrypt(binary_data)

    # Save the encrypted file locally
    binary_to_file(encrypted_data, encrypted_output_path)

    # Upload encrypted file to Google Drive
    gauth = GoogleAuth()
    gauth.LocalWebserverAuth()  # Authenticate with the user's credentials
    drive = GoogleDrive(gauth)
    file_drive = drive.CreateFile({'title': encrypted_output_path})
    file_drive.SetContentFile(encrypted_output_path)
    file_drive.Upload()

    # Log blockchain data
    device_ip = socket.gethostbyname(socket.gethostname())
    block_data = {
        "block_id": secrets.token_hex(8),
        "timestamp": datetime.now().isoformat(),
        "file_name": encrypted_output_path,
        "device_ip": device_ip,
        "operation": "encryption"
    }
    save_blockchain_data(block_data)

    print(f"File encrypted successfully! Encrypted file: {encrypted_output_path}")
    print(f"Hash Key (Save this securely!): {hash_key}")

if __name__ == "__main__":
    # Get user input for the file path
    file_path = input("Enter the path of the file to encrypt: ").strip()
    encrypted_output_path = input("Enter the path to save the encrypted file (e.g., encrypted.dat): ").strip()

    # Encrypt the file
    encrypt_file(file_path, encrypted_output_path)


InvalidConfigError: Invalid client secrets file ('Error opening file', 'client_secrets.json', 'No such file or directory', 2)

In [9]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
import os
import json
from datetime import datetime
import socket

# Authenticate and connect to Google Drive
def authenticate_google_drive():
    gauth = GoogleAuth()
    # Try to load saved client credentials
    gauth.LoadCredentialsFile("mycreds.txt")
    if gauth.credentials is None:
        # Authenticate if they're not there
        gauth.LocalWebserverAuth()
    elif gauth.access_token_expired:
        # Refresh credentials if expired
        gauth.Refresh()
    else:
        # Initialize the saved credentials
        gauth.Authorize()
    # Save the current credentials to a file
    gauth.SaveCredentialsFile("mycreds.txt")
    return GoogleDrive(gauth)

# Upload a file to Google Drive
def upload_file_to_drive(drive, file_path, file_name):
    file = drive.CreateFile({'title': file_name})
    file.SetContentFile(file_path)
    file.Upload()
    print(f"Uploaded {file_name} to Google Drive.")
    return file['id']

# Save blockchain data to a JSON file
def save_blockchain_data(data):
    """Save blockchain data to a JSON file."""
    try:
        with open("block_data.json", "r") as file:
            blockchain = json.load(file)
    except FileNotFoundError:
        print("Blockchain data file not found. Creating a new block_data.json file.")
        blockchain = []

    blockchain.append(data)
    with open("block_data.json", "w") as file:
        json.dump(blockchain, file, indent=4)

# Main workflow
if __name__ == "__main__":
    # Authenticate Google Drive
    drive = authenticate_google_drive()

    # File to upload
    file_path = input("Enter the path of the file to upload: ").strip()
    file_name = os.path.basename(file_path)

    # Upload the file to Google Drive
    file_id = upload_file_to_drive(drive, file_path, file_name)

    # Save blockchain metadata
    blockchain_data = {
        "block_id": len(open("block_data.json").readlines()) + 1 if os.path.exists("block_data.json") else 1,
        "timestamp": str(datetime.now()),
        "file_name": file_name,
        "file_id": file_id,
        "device_ip": socket.gethostbyname(socket.gethostname()),
        "action": "upload"
    }
    save_blockchain_data(blockchain_data)
    print("Blockchain metadata saved.")


/home/wac/.local/lib/python3.10/site-packages/oauth2client/_helpers.py:255: UserWarning: Cannot access mycreds.txt: No such file or directory
  warnings.warn(_MISSING_FILE_MESSAGE.format(filename))


InvalidConfigError: Invalid client secrets file ('Error opening file', 'client_secrets.json', 'No such file or directory', 2)

independent decrypter

In [ ]:
import base64
import hashlib
import secrets
import json
import socket
from cryptography.fernet import Fernet
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from datetime import datetime

# Generate a random hash key
def generate_random_hash_key():
    """Generate a random hash key using secrets."""
    return secrets.token_hex(16)

# Generate an encryption key from a hash key
def generate_encryption_key(hash_key):
    """Generate a SHA-256 encryption key from the hash key."""
    sha256 = hashlib.sha256()
    sha256.update(hash_key.encode('utf-8'))
    return base64.urlsafe_b64encode(sha256.digest())

# Convert file to binary
def file_to_binary(file_path):
    """Convert any file to binary data."""
    with open(file_path, 'rb') as file:
        return file.read()

# Convert binary to file
def binary_to_file(binary_data, output_path):
    """Convert binary data back to a file."""
    with open(output_path, 'wb') as file:
        file.write(binary_data)

# Save blockchain data to JSON file
def save_blockchain_data(data):
    """Save blockchain data to a JSON file."""
    try:
        with open("block_data.json", "r") as file:
            blockchain = json.load(file)
    except FileNotFoundError:
        blockchain = []

    blockchain.append(data)
    with open("block_data.json", "w") as file:
        json.dump(blockchain, file, indent=4)

# Encrypt a file and save the encrypted version
def encrypt_file(file_path, encrypted_output_path):
    """Encrypt the file and save it."""
    # Generate hash key and encryption key
    hash_key = generate_random_hash_key()
    encryption_key = generate_encryption_key(hash_key)

    # Read and encrypt the file
    binary_data = file_to_binary(file_path)
    fernet = Fernet(encryption_key)
    encrypted_data = fernet.encrypt(binary_data)

    # Save the encrypted file locally
    binary_to_file(encrypted_data, encrypted_output_path)

    # Upload encrypted file to Google Drive
    gauth = GoogleAuth()
    gauth.LocalWebserverAuth()  # Authenticate with the user's credentials
    drive = GoogleDrive(gauth)
    file_drive = drive.CreateFile({'title': encrypted_output_path})
    file_drive.SetContentFile(encrypted_output_path)
    file_drive.Upload()

    # Log blockchain data
    device_ip = socket.gethostbyname(socket.gethostname())
    block_data = {
        "block_id": secrets.token_hex(8),
        "timestamp": datetime.now().isoformat(),
        "file_name": encrypted_output_path,
        "device_ip": device_ip,
        "operation": "encryption"
    }
    save_blockchain_data(block_data)

    print(f"File encrypted successfully! Encrypted file: {encrypted_output_path}")
    print(f"Hash Key (Save this securely!): {hash_key}")

if __name__ == "__main__":
    # Get user input for the file path
    file_path = input("Enter the path of the file to encrypt: ").strip()
    encrypted_output_path = input("Enter the path to save the encrypted file (e.g., encrypted.dat): ").strip()

    # Encrypt the file
    encrypt_file(file_path, encrypted_output_path)


In [10]:
import base64
import hashlib
import secrets
import json
import socket
from cryptography.fernet import Fernet
from datetime import datetime
import os

# Generate a random hash key
def generate_random_hash_key():
    """Generate a random hash key using secrets."""
    return secrets.token_hex(16)

# Generate an encryption key from a hash key
def generate_encryption_key(hash_key):
    """Generate a SHA-256 encryption key from the hash key."""
    sha256 = hashlib.sha256()
    sha256.update(hash_key.encode('utf-8'))
    return base64.urlsafe_b64encode(sha256.digest())

# Convert file to binary
def file_to_binary(file_path):
    """Convert any file to binary data."""
    with open(file_path, 'rb') as file:
        return file.read()

# Convert binary to file
def binary_to_file(binary_data, output_path):
    """Convert binary data back to a file."""
    with open(output_path, 'wb') as file:
        file.write(binary_data)

# Save blockchain data to JSON file
def save_blockchain_data(data):
    """Save blockchain data to a JSON file."""
    try:
        with open("block_data.json", "r") as file:
            blockchain = json.load(file)
    except FileNotFoundError:
        blockchain = []

    blockchain.append(data)
    with open("block_data.json", "w") as file:
        json.dump(blockchain, file, indent=4)

# Encrypt a file and save the encrypted version
def encrypt_file(file_path, encrypted_output_path):
    """Encrypt the file and save it."""
    # Generate hash key and encryption key
    hash_key = generate_random_hash_key()
    encryption_key = generate_encryption_key(hash_key)

    # Read and encrypt the file
    binary_data = file_to_binary(file_path)
    fernet = Fernet(encryption_key)
    encrypted_data = fernet.encrypt(binary_data)

    # Save the encrypted file locally
    binary_to_file(encrypted_data, encrypted_output_path)

    # Log blockchain data locally
    device_ip = socket.gethostbyname(socket.gethostname())
    block_data = {
        "block_id": secrets.token_hex(8),
        "timestamp": datetime.now().isoformat(),
        "file_name": os.path.basename(encrypted_output_path),
        "file_path": os.path.abspath(encrypted_output_path),
        "device_ip": device_ip,
        "operation": "encryption"
    }
    save_blockchain_data(block_data)

    print(f"File encrypted successfully! Encrypted file: {encrypted_output_path}")
    print(f"Hash Key (Save this securely!): {hash_key}")

if __name__ == "__main__":
    # Get user input for the file path
    file_path = input("Enter the path of the file to encrypt: ").strip()
    encrypted_output_path = input("Enter the path to save the encrypted file (e.g., encrypted.dat): ").strip()

    # Encrypt the file
    encrypt_file(file_path, encrypted_output_path)


File encrypted successfully! Encrypted file: blockdata.dat
Hash Key (Save this securely!): 31c7fcd20f12a9f475894fcbeb96aa41


In [14]:
# working encrypter

import base64
import hashlib
import secrets
import json
import socket
from cryptography.fernet import Fernet
from datetime import datetime
import os

# Generate a random hash key
def generate_random_hash_key():
    """Generate a random hash key using secrets."""
    return secrets.token_hex(16)

# Generate an encryption key from a hash key
def generate_encryption_key(hash_key):
    """Generate a SHA-256 encryption key from the hash key."""
    sha256 = hashlib.sha256()
    sha256.update(hash_key.encode('utf-8'))
    return base64.urlsafe_b64encode(sha256.digest())

# Convert file to binary
def file_to_binary(file_path):
    """Convert any file to binary data."""
    with open(file_path, 'rb') as file:
        return file.read()

# Convert binary to file
def binary_to_file(binary_data, output_path):
    """Convert binary data back to a file."""
    with open(output_path, 'wb') as file:
        file.write(binary_data)

# Save blockchain data to JSON file
def save_blockchain_data(data):
    """Save blockchain data to a JSON file."""
    try:
        with open("block_data.json", "r") as file:
            blockchain = json.load(file)
    except FileNotFoundError:
        blockchain = []

    blockchain.append(data)
    with open("block_data.json", "w") as file:
        json.dump(blockchain, file, indent=4)

# Encrypt a file and save the encrypted version
def encrypt_file(file_path, encrypted_output_path):
    """Encrypt the file and save it."""
    # Generate hash key and encryption key
    hash_key = generate_random_hash_key()
    encryption_key = generate_encryption_key(hash_key)

    # Read and encrypt the file
    binary_data = file_to_binary(file_path)
    fernet = Fernet(encryption_key)
    encrypted_data = fernet.encrypt(binary_data)

    # Save the encrypted file locally
    binary_to_file(encrypted_data, encrypted_output_path)

    # Log blockchain data locally
    device_ip = socket.gethostbyname(socket.gethostname())
    block_data = {
        "block_id": secrets.token_hex(8),
        "timestamp": datetime.now().isoformat(),
        "file_name": encrypted_output_path,
        "file_path": os.path.abspath(encrypted_output_path),
        "device_ip": device_ip,
        "operation": "encryption"
    }
    save_blockchain_data(block_data)

    print(f"File encrypted successfully! Encrypted file: {encrypted_output_path}")
    print(f"Hash Key (Save this securely!): {hash_key}")

if __name__ == "__main__":
    # Get user input for the file path
    file_path = input("Enter the path of the file to encrypt: ").strip()
    encrypted_output_path = input("Enter the path to save the encrypted file (e.g., encrypted.dat): ").strip()

    # Encrypt the file
    encrypt_file(file_path, encrypted_output_path)


File encrypted successfully! Encrypted file: sample_data_encrypted.dat
Hash Key (Save this securely!): cda2284573d10ebf4b700c06e5e6a72f


In [15]:
# working decrypter

import base64
import hashlib
import secrets
import json
import socket
from cryptography.fernet import Fernet
from datetime import datetime
import os

# Generate an encryption key from a hash key
def generate_encryption_key(hash_key):
    """Generate a SHA-256 encryption key from the hash key."""
    sha256 = hashlib.sha256()
    sha256.update(hash_key.encode('utf-8'))
    return base64.urlsafe_b64encode(sha256.digest())

# Convert file to binary
def file_to_binary(file_path):
    """Convert any file to binary data."""
    with open(file_path, 'rb') as file:
        return file.read()

# Convert binary to file
def binary_to_file(binary_data, output_path):
    """Convert binary data back to a file."""
    with open(output_path, 'wb') as file:
        file.write(binary_data)

# Save blockchain data to JSON file
def save_blockchain_data(data):
    """Save blockchain data to a JSON file."""
    try:
        with open("block_data.json", "r") as file:
            blockchain = json.load(file)
    except FileNotFoundError:
        blockchain = []

    blockchain.append(data)
    with open("block_data.json", "w") as file:
        json.dump(blockchain, file, indent=4)

# Decrypt a file and save the decrypted version
def decrypt_file(encrypted_file_path, decrypted_output_path, hash_key):
    """Decrypt the file and save it."""
    # Generate encryption key from the provided hash key
    encryption_key = generate_encryption_key(hash_key)

    # Read and decrypt the file
    encrypted_data = file_to_binary(encrypted_file_path)
    fernet = Fernet(encryption_key)
    decrypted_data = fernet.decrypt(encrypted_data)

    # Save the decrypted file locally
    binary_to_file(decrypted_data, decrypted_output_path)

    # Log blockchain data locally
    device_ip = socket.gethostbyname(socket.gethostname())
    block_data = {
        "block_id": secrets.token_hex(8),
        "timestamp": datetime.now().isoformat(),
        "file_name": decrypted_output_path,
        "file_path": os.path.abspath(decrypted_output_path),
        "device_ip": device_ip,
        "operation": "decryption"
    }
    save_blockchain_data(block_data)

    print(f"File decrypted successfully! Decrypted file: {decrypted_output_path}")

if __name__ == "__main__":
    # Get user input for the encrypted file path and hash key
    encrypted_file_path = input("Enter the path of the encrypted file: ").strip()
    decrypted_output_path = input("Enter the path to save the decrypted file (e.g., decrypted.dat): ").strip()
    hash_key = input("Enter the hash key for decryption: ").strip()

    # Decrypt the file
    decrypt_file(encrypted_file_path, decrypted_output_path, hash_key)


File decrypted successfully! Decrypted file: sample_decrypted.xlsx
